# AmebaPro2 LLM Whisper HTTP Server
## with pyNgrok tunnel

In [1]:
!pip install git+https://github.com/openai/whisper.git
!pip install fastapi uvicorn
!pip install pyngrok
!pip install accelerate
!pip install einops
!pip install nest-asyncio
!pip install python-multipart

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-4e9itu84
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-4e9itu84
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-

In [2]:
import whisper
ASR = whisper.load_model("base").to("cpu")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 108MiB/s]


## LLM model

In [3]:
import torch
import transformers
from transformers import AutoModelForCausalLM , AutoTokenizer

### https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
model_name = "Q-bert/Mamba-130M"
#model_name = "Q-bert/Mamba-370M"
#model_name = "Q-bert/Mamba-790M"
#model_name = "Q-bert/Mamba-1B"
#model_name = "Q-bert/Mamba-3B"
#model_name = "Q-bert/Mamba-3B-slimpj"
#model_name = "ckip-joint/bloom-3b-zh" # zh
#model_name = "google/gemma-2b-it"
#model_name = "microsoft/phi-2"
#model_name = "Qwen/Qwen1.5-7B-Chat" # cn
#model_name = "lmsys/vicuna-7b-v1.5-16k" # zh/cn
#model_name = "yentinglin/Taiwan-LLM-7B-v2.0.1-chat" # zh
#model_name = "mistralai/Mistral-7B-Instruct-v0.2"
#model_name = "MediaTek-Research/Breeze-7B-Instruct-v0.1" # zh/cn

LLM = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype="auto", device_map="cuda") # for Mamba
#LLM = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.bfloat16, device_map="cuda") # for the rest models

tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/517 [00:00<?, ?B/s]

configuration_mamba.py:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Q-bert/Mamba-130M:
- configuration_mamba.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_mamba.py:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/457k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## HTTP Server with Ngrok

In [4]:
import getpass
import os
import threading

from pyngrok import ngrok, conf


## set ngrok authtoken
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········


In [5]:
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


 * ngrok tunnel "https://067a-35-230-19-44.ngrok-free.app" -> "http://127.0.0.1:5000/"


In [6]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import Response
from pydantic import BaseModel
import uvicorn
import json

app = FastAPI()

class Base64Data(BaseModel):
    base64_string: str

@app.get("/")
def root():
    return Response("Hello World!")

@app.post("/audio")
def post_audio(data: Base64Data ):
    try:
        #Decode the base64 string
        decoded_data = base64.b64decode(data.base64_string)

        # print(decoded_data)
        #Save the decoded data to an MP4 file
        with open("output.mp4", "wb") as f:
            f.write(decoded_data)

        # Whisper transcribe
        result = ASR.transcribe("output.mp4")
        print("ASR: "+result["text"])

        # LLM generate
        prompt = result["text"]
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        output = LLM.generate(input_ids, max_length=128, num_beams=5, no_repeat_ngram_size=2)
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        print("LLM: "+generated_text)
        return Response(generated_text)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


# start new thread
threading.Thread(uvicorn.run(app, host="127.0.0.1", port=5000, log_level="info")).start()

INFO:     Started server process [1176]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     220.130.190.245:0 - "GET / HTTP/1.1" 200 OK
INFO:     220.130.190.245:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
